# Ingesta histórica + incremental de series macroeconómicas FRED hacia Azure Data Lake

**Descripcion:**<br>
Este script en Python consume la API de la Reserva Federal (fredapi) para descargar observaciones mensuales/diarias de seis indicadores clave (FEDFUNDS, CPI, desempleo, PIB real, Treasury 10 años y Financial Stress Index) desde 1992 hasta hoy.<br>
Los datos se normalizan, validan y se guardan en formato Parquet local (zona RAW) para después replicarse y particionarse por año en Azure Data Lake Storage Gen2 (zona Bronze). Incluye controles básicos de calidad, bitácora de ingesta y lógica incremental para futuras actualizaciones.<br>

**Objetivos**:<br>
-Automatizar la extracción de series macroeconómicas desde la API de FRED.<br>
-Estandarizar el esquema.<br>
-Persistir en formato eficiente (Parquet) tanto localmente como en ADLS, habilitando compresión y lectura columnar.<br>
-Garantizar la calidad inicial mediante chequeos de cobertura, nulos y duplicados.<br>
-Implementar carga incremental idempotente, particionando por year=YYYY.<br>
-Registrar la actividad (log de ingesta y tamaños de archivo).

## 1. Configuración básica y librerías y Parámetros globales del pipeline

In [1]:
# ================================================================
# 1.1  Librerías estándar y de terceros, así como funciones propias
# ================================================================
import os, io, pathlib, logging, datetime as dt
from datetime import date

import pandas as pd
from fredapi import Fred                                     # cliente oficial de la API FRED
from dotenv import load_dotenv                               # Leer variables de entorno

# Funciones auxiliares para Azure Data Lake Storage (ya desarrolladas)
from utils_adls import upload_bytes, _client, CONTAINER      # funciones ya creadas

load_dotenv()                 # lee .env  (FRED_API_KEY, AZ_STORAGE_ACCOUNT)

# --- parámetros generales ---------------------------------------
FRED_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_KEY)         # instancia del cliente FRED

# Diccionario {código FRED: descripción legible}
SERIES = {
    # Base (mantienes las tuyas; sugiero usar DGS10 en lugar de GS10 por ser diaria)
    "FEDFUNDS":  "Effective Fed Funds Rate",
    "CPIAUCSL":  "CPI (SA)",
    "UNRATE":    "Unemployment Rate",
    "GDP":       "Real GDP",
    "DGS10":     "10-Year Treasury Constant Maturity (Daily)",  # reemplaza a GS10
    "STLFSI2":   "St. Louis Fed Financial Stress Index (v2)",

    # Curva/expectativas
    "DGS2":      "2-Year Treasury Constant Maturity",
    "T10Y3M":    "10Y Minus 3M Treasury Spread",
    "T10Y2Y":    "10Y Minus 2Y Treasury Spread",
    "DFII10":    "10-Year TIPS Yield (Real)",
    "T5YIE":     "5-Year Breakeven Inflation",
    "THREEFYTP10": "Kim-Wright 10Y Zero-Coupon Term Premium",  # en lugar de ACMTP10

    # Crédito / estrés
    "BAMLH0A0HYM2": "ICE BofA US High Yield Index Option-Adjusted Spread",  # en lugar de ...OAS
    "BAA10Y":       "Moody's BAA Corporate - 10Y Treasury Spread",
    "NFCI":         "Chicago Fed National Financial Conditions Index",
    "ANFCI":        "Adjusted NFCI",
    "TEDRATE":      "TED Spread",

    # Volatilidad y risk-on/off
    "VIXCLS":    "CBOE VIX Close",

    # Dólar y commodities
    "DTWEXBGS":  "Trade-Weighted U.S. Dollar Index: Broad",
    "DCOILWTICO":"Crude Oil WTI Spot Price",

    # Actividad / empleo / sentimiento
    "INDPRO":    "Industrial Production Index",
    "ICSA":      "Initial Claims, Unemployment Insurance (Weekly)",
    "PAYEMS":    "All Employees: Total Nonfarm",
    "UMCSENT":   "University of Michigan: Sentiment",

    # Vivienda
    "HOUST":     "Housing Starts: Total",
    "PERMIT":    "Building Permits: Total",

    # Liquidez / balance Fed
    "M2SL":      "M2 Money Stock",
    "WALCL":     "Total Assets, Federal Reserve"
}


START_DATE = "1992-01-01"         # fecha inicial para todas las series
END_DATE   = date.today().isoformat()  # fecha final dinámica (hoy)

# Rutas locales y prefijos en ADLS ---------------------------
RAW_DIR       = "data/raw/macros";  os.makedirs(RAW_DIR, exist_ok=True)
BRONZE_PREFIX = "bronze/macros"

## 2. Descarga histórica, subida a ADLS y Chequeos rapido

In [2]:
# ================================================================
# 2.1  descarga histórica y subida a ADLS
# ================================================================

# -------------------------------------------------------------------------------
# - Descarga cada serie completa usando fredapi.
# - Guarda un Parquet en la carpeta RAW local.
# - Sube la misma copia a ADLS bajo raw/macros/.
# -------------------------------------------------------------------------------
def download_series(code: str) -> pd.DataFrame:
    ser = fred.get_series(code, observation_start=START_DATE, observation_end=END_DATE)
    df  = ser.to_frame("value").reset_index().rename(columns={"index": "date"})
    df["series"] = code
    return df

for code in SERIES:
    print("⏬", code)
    df = download_series(code)

    # 1a) guarda RAW local
    local_raw = f"{RAW_DIR}/{code}.parquet"
    df.to_parquet(local_raw, index=False)

    # 1b) sube RAW a ADLS
    with open(local_raw, "rb") as f:
        upload_bytes(f"raw/macros/{code}.parquet", f.read())

⏬ FEDFUNDS
⏬ CPIAUCSL
⏬ UNRATE
⏬ GDP
⏬ DGS10
⏬ STLFSI2
⏬ DGS2
⏬ T10Y3M
⏬ T10Y2Y
⏬ DFII10
⏬ T5YIE
⏬ THREEFYTP10
⏬ BAMLH0A0HYM2
⏬ BAA10Y
⏬ NFCI
⏬ ANFCI
⏬ TEDRATE
⏬ VIXCLS
⏬ DTWEXBGS
⏬ DCOILWTICO
⏬ INDPRO
⏬ ICSA
⏬ PAYEMS
⏬ UMCSENT
⏬ HOUST
⏬ PERMIT
⏬ M2SL
⏬ WALCL


In [3]:
# ================================================================
# 2.2  Chequeos rapidos (EDA)
# ================================================================

# ----------------------------------------------------------------
# Combina todos los Parquet RAW y realiza:
#   - Cobertura temporal por serie (fecha mín, máx y nº observaciones).
#   - Porcentaje de valores nulos.
#   - Tamaño físico de cada archivo.
# Estos chequeos se imprimen / muestran en el notebook para validación inmediata.
# -----------------------------------------------------------------

dfs = [pd.read_parquet(f"{RAW_DIR}/{c}.parquet") for c in SERIES]
macro_df = pd.concat(dfs)

#  Cobertura temporal ---------------------------------------------
coverage = (
    macro_df.groupby("series")["date"]
            .agg(fecha_min=("min"), fecha_max=("max"), num_obs=("count"))
)
display(coverage)

# Valores nulos ---------------------------------------------------
nulls_pct = (macro_df.isna().mean() * 100).round(2)
print("NaNs (%):\n", nulls_pct)

# Tamaño de archivos ----------------------------------------------
sizes = {p.name: round(p.stat().st_size / 1_048_576, 2)
         for p in pathlib.Path(RAW_DIR).glob("*.parquet")}
print("Tamaños (MB):", sizes)

,fecha_min,fecha_max,num_obs
series,,,
ANFCI,1992-01-03,2025-07-18,1751
BAA10Y,1992-01-01,2025-07-24,8757
BAMLH0A0HYM2,1996-12-31,2025-07-24,7549
CPIAUCSL,1992-01-01,2025-06-01,402
DCOILWTICO,1992-01-01,2025-07-21,8754
DFII10,2003-01-02,2025-07-24,5886
DGS10,1992-01-01,2025-07-24,8757
DGS2,1992-01-01,2025-07-24,8757
DTWEXBGS,2006-01-02,2025-07-18,5100


NaNs (%):
 date      0.00
value     3.55
series    0.00
dtype: float64
Tamaños (MB): {'ANFCI.parquet': 0.03, 'BAA10Y.parquet': 0.09, 'BAMLH0A0HYM2.parquet': 0.08, 'CPIAUCSL.parquet': 0.01, 'DCOILWTICO.parquet': 0.11, 'DFII10.parquet': 0.06, 'DGS10.parquet': 0.09, 'DGS2.parquet': 0.09, 'DTWEXBGS.parquet': 0.08, 'FEDFUNDS.parquet': 0.01, 'GDP.parquet': 0.0, 'HOUST.parquet': 0.01, 'ICSA.parquet': 0.02, 'INDPRO.parquet': 0.01, 'M2SL.parquet': 0.01, 'NFCI.parquet': 0.03, 'PAYEMS.parquet': 0.01, 'PERMIT.parquet': 0.01, 'STLFSI2.parquet': 0.03, 'T10Y2Y.parquet': 0.09, 'T10Y3M.parquet': 0.09, 'T5YIE.parquet': 0.06, 'TEDRATE.parquet': 0.08, 'THREEFYTP10.parquet': 0.13, 'UMCSENT.parquet': 0.01, 'UNRATE.parquet': 0.01, 'VIXCLS.parquet': 0.1, 'WALCL.parquet': 0.02}


# Etapa Bronze

## 3. Ingesta inicial (histórico completo) -> Azure Data Lake Storage y registro en Logs

In [4]:
# ================================================================
# 3.1 Ingesta inicial en Bronze, solo se debe ejecutar una vez como carga inicial
# ================================================================

# ----------------------------------------------------------------
# Convierte el histórico descargado en la *capa Bronze*:
#   • Añade columnas de metadatos (`year`, `ingest_ts`, `source`).
#   • Particiona por año (year=YYYY) y sube cada partición a ADLS.
#   • Este bloque debe correrse una sola vez para poblar todo el histórico.
# ---------------------------------------------------------------

for code in SERIES:
    df = pd.read_parquet(f"{RAW_DIR}/{code}.parquet")
    
    # Agrega columnas de metadatos ------------------------------
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df["ingest_ts"] = pd.Timestamp.utcnow().isoformat()
    df["source"]    = "FRED"

     # Sube cada año como Parquet independiente -----------------
    for yr, sub in df.groupby("year"):
        buf = io.BytesIO(); sub.to_parquet(buf, index=False); buf.seek(0)
        remote = f"{BRONZE_PREFIX}/{code}/year={yr}/{code}_{yr}.parquet"
        upload_bytes(remote, buf.read())                 # overwrite=True
        print(f"▲ {code} {yr} -> {remote}  ({len(sub)} filas)")

▲ FEDFUNDS 1992 -> bronze/macros/FEDFUNDS/year=1992/FEDFUNDS_1992.parquet  (12 filas)
▲ FEDFUNDS 1993 -> bronze/macros/FEDFUNDS/year=1993/FEDFUNDS_1993.parquet  (12 filas)
▲ FEDFUNDS 1994 -> bronze/macros/FEDFUNDS/year=1994/FEDFUNDS_1994.parquet  (12 filas)
▲ FEDFUNDS 1995 -> bronze/macros/FEDFUNDS/year=1995/FEDFUNDS_1995.parquet  (12 filas)
▲ FEDFUNDS 1996 -> bronze/macros/FEDFUNDS/year=1996/FEDFUNDS_1996.parquet  (12 filas)
▲ FEDFUNDS 1997 -> bronze/macros/FEDFUNDS/year=1997/FEDFUNDS_1997.parquet  (12 filas)
▲ FEDFUNDS 1998 -> bronze/macros/FEDFUNDS/year=1998/FEDFUNDS_1998.parquet  (12 filas)
▲ FEDFUNDS 1999 -> bronze/macros/FEDFUNDS/year=1999/FEDFUNDS_1999.parquet  (12 filas)
▲ FEDFUNDS 2000 -> bronze/macros/FEDFUNDS/year=2000/FEDFUNDS_2000.parquet  (12 filas)
▲ FEDFUNDS 2001 -> bronze/macros/FEDFUNDS/year=2001/FEDFUNDS_2001.parquet  (12 filas)
▲ FEDFUNDS 2002 -> bronze/macros/FEDFUNDS/year=2002/FEDFUNDS_2002.parquet  (12 filas)
▲ FEDFUNDS 2003 -> bronze/macros/FEDFUNDS/year=2003/FE

In [5]:
# ================================================================
# 3.1 Log de Ingesta
# ================================================================
LOG_DIR = "logs"; os.makedirs(LOG_DIR, exist_ok=True)
logging.basicConfig(
    filename=f"{LOG_DIR}/ingest_macros_{dt.date.today()}.log",
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)
for name, mb in sizes.items():
    logging.info(f"{name}: {mb} MB guardado en RAW")
logging.info("Cobertura: %s", coverage.to_dict())

## 4. Carga incremental de último año cargado

In [6]:
# ================================================================
# 4.1 Carga Incremental (Para ejecuciones programadas)
# ================================================================

# ----------------------------------------------------------------
# Detecta la última fecha cargada en Bronze y descarga sólo las observaciones
# nuevas desde esa fecha +1 hasta hoy.  Idempotente: si no hay datos nuevos,
# la serie se salta.
# ----------------------------------------------------------------
svc = _client()

"""Devuelve la última fecha disponible en Bronze para la serie dada."""
def last_date(series: str):
    d = svc.get_directory_client(CONTAINER, f"{BRONZE_PREFIX}/{series}")
    years = sorted(int(p.name.split('=')[1]) for p in d.get_paths() if p.is_directory)
    if not years: return None
    latest_year = years[-1]
    latest_file = next(p.name for p in d.get_paths()
                       if (not p.is_directory) and f"year={latest_year}/" in p.name)
    raw = svc.get_file_client(CONTAINER, latest_file).download_file().readall()
    return pd.read_parquet(io.BytesIO(raw), columns=["date"])["date"].max().date()

# ► Descarga incremental por serie -------------------------------
today = dt.date.today()
for code in SERIES:
    ld = last_date(code)
    start = (ld + dt.timedelta(days=1)) if ld else pd.to_datetime(START_DATE).date()
    if start > today:
        print(code, "al día"); continue

    # Descarga desde la fecha 'start' hasta hoy ------------------
    df_new = fred.get_series(code, observation_start=start, observation_end=today).to_frame("value")
    if df_new.empty:
        continue
    
    # Normaliza y añade metadatos --------------------------------
    df_new = (df_new.reset_index()
                     .rename(columns={"index": "date"})
                     .assign(series=code,
                             year=lambda x: x["date"].dt.year,
                             ingest_ts=pd.Timestamp.utcnow().isoformat(),
                             source="FRED"))

    # Sube particiones anuales nuevas o sobrescribe existente ----
    for yr, part in df_new.groupby("year"):
        buf = io.BytesIO(); part.to_parquet(buf, index=False); buf.seek(0)
        remote = f"{BRONZE_PREFIX}/{code}/year={yr}/{code}_{yr}.parquet"
        upload_bytes(remote, buf.read())
        print(f"▲ {code} {yr} +{len(part)} obs")
print("\nProceso completado ✅")

▲ FEDFUNDS 2025 +1 obs
▲ CPIAUCSL 2025 +1 obs
▲ UNRATE 2025 +1 obs
▲ GDP 2025 +1 obs
▲ INDPRO 2025 +1 obs
▲ PAYEMS 2025 +1 obs
▲ UMCSENT 2025 +1 obs
▲ HOUST 2025 +1 obs
▲ PERMIT 2025 +1 obs
▲ M2SL 2025 +1 obs

Proceso completado ✅
